# Position‑Sizing Strategies

In algorithmic trading, **position sizing** determines how much capital to commit to each signal.
Proper sizing controls drawdowns, preserves capital, and maximises risk‑adjusted returns.

Methods

1. Equally‑Weighted  
2. Volatility‑Scaled  
3. Notional‑Target  
4. Fixed‑Fractional  
5. Risk‑Parity  
6. Kelly Criterion  
7. Maximum Drawdown Control

---

In [17]:
# 1. Imports & Dummy Data
import numpy as np
import pandas as pd

# load real price data
prices = pd.read_csv(
    "/Users/mchildress/Active Code/ts_basics/data/bnbusdt_1m.csv",
    index_col="timestamp", parse_dates=True
)[["close"]]
# example: simple momentum signal (+1 if price up, -1 if down, 0 otherwise)
rets = prices["close"].pct_change()
signals = pd.DataFrame(0, index=prices.index, columns=prices.columns)
signals.loc[rets > 0, "close"] = 1
signals.loc[rets < 0, "close"] = -1

nav = 1_000_000

/var/folders/yj/3s0hc5nn3qlg4lqp7wmfgq_c0000gn/T/ipykernel_2891/4107573104.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices = pd.read_csv(


## 1. Equally‑Weighted

Divide NAV equally across all **currently signaled** instruments.

- **Pros**: trivial, basic diversification  
- **Cons**: ignores different volatilities
- Capital is divided equally across each trade so every position gets the same budget, without worrying about which one moves more.

In [18]:
def eq_weighted(signals, prices, nav):
    npos = signals.replace(0, np.nan).count(axis=1)
    alloc = nav / npos
    pos = (signals.mul(alloc, axis=0) / prices).fillna(0).astype(int)
    return pos

eq_pos = eq_weighted(signals, prices, nav)
eq_pos

,close
timestamp,
1514764800,0
1514764860,0
1514764920,-117446
1514764980,-117857
1514765040,117555
...,...
1735775700,0
1735775760,1412
1735775820,0


## 2. Volatility‑Scaled

Target a constant **portfolio volatility** by scaling each position inverse to its recent σ.

- **Pros**: balances risk contributions  
- **Cons**: needs reliable σ estimates
- Investment is made in assets that move smoothly and less in those that jump around, keeping our overall portfolio swings under control.

In [19]:
def vol_scaled(signals, prices, nav, target_vol=0.10):
    rets = prices.pct_change().dropna()
    vol = rets.rolling(5).std().reindex(signals.index).fillna(method="bfill")
    scale = (target_vol / vol).clip(upper=1.0)
    alloc = signals.mul(nav * scale, axis=0)
    return (alloc / prices).fillna(0).astype(int)

vs_pos = vol_scaled(signals, prices, nav)
vs_pos

/var/folders/yj/3s0hc5nn3qlg4lqp7wmfgq_c0000gn/T/ipykernel_2891/146314018.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vol = rets.rolling(5).std().reindex(signals.index).fillna(method="bfill")


,close
timestamp,
1514764800,0
1514764860,0
1514764920,-117446
1514764980,-117857
1514765040,117555
...,...
1735775700,0
1735775760,1412
1735775820,0


## 3. Notional‑Target

Allocate a fixed % of NAV to each signal, e.g. 10%.

- **Pros**: clear cap per position  
- **Cons**: ignores volatility
- A fixed share of our capital is set for each position—like always using 10% of our total funds per trade.

In [20]:
def notional_target(signals, prices, nav, max_pct=0.10):
    alloc = signals * nav * max_pct
    return (alloc / prices).fillna(0).astype(int)

nt_pos = notional_target(signals, prices, nav)
nt_pos

,close
timestamp,
1514764800,0
1514764860,0
1514764920,-11744
1514764980,-11785
1514765040,11755
...,...
1735775700,0
1735775760,141
1735775820,0


## 4. Fixed‑Fractional

Risk a constant fraction of NAV per trade, sizing via stop‑loss distance.

- **Pros**: consistent £‑at‑risk  
- **Cons**: needs stop levels
- Pick a dollar amount we’re willing to risk, then calculate position size so that if the trade hits our stop‑loss, we lose only that set amount.

In [21]:
# dummy stop levels at 2% below price
stops = prices * 0.98

def fixed_fraction(signals, prices, stops, nav, risk_pct=0.02):
    risk_amt = nav * risk_pct
    size = (risk_amt / (prices - stops)).fillna(0)
    return size.astype(int)

ff_pos = fixed_fraction(signals, prices, stops, nav)
ff_pos

,close
timestamp,
1514764800,117233
1514764860,117233
1514764920,117446
1514764980,117857
1514765040,117555
...,...
1735775700,1412
1735775760,1412
1735775820,1412


## 5. Risk‑Parity

Solve for weights so each asset contributes equally to portfolio variance.

- **Pros**: maximises diversification  
- **Cons**: requires covariance & optimisation
- Risk is spread evenly so no single asset dominates the portfolio’s ups and downs.

In [22]:
import scipy.optimize as opt

def risk_parity(signals, prices, nav):
    rets = prices.pct_change().dropna()
    Σ = rets.cov()
    n = Σ.shape[0]
    w0 = np.ones(n) / n
    def obj(w):
        σp = np.sqrt(w @ Σ @ w)
        mrc = (Σ @ w) / σp
        rc = w * mrc
        return ((rc - σp/n) ** 2).sum()
    cons = ({'type':'eq','fun':lambda w: w.sum()-1})
    bnds = [(0,1)]*n
    w = opt.minimize(obj, w0, bounds=bnds, constraints=cons).x
    alloc = pd.Series(w, index=prices.columns) * nav
    return (alloc / prices.iloc[-1]).astype(int)

rp_pos = risk_parity(signals, prices, nav)
rp_pos

close    1412
dtype: int64

## 6. Kelly Criterion

Maximise growth by sizing $f* = (p(b+1) - 1)/b$, where b=edge, p=win%.

- **Pros**: long‑term growth  
- **Cons**: parameter‑sensitive
- Based on historical win rate and average win size, this method tells us how much of our capital to risk each time to maximize long-term growth.

In [23]:
# scalar expected return & win‑prob
exp_r = 0.002   # 0.2% edge
p_win = 0.60   # 60% win rate

def kelly(signals, prices, nav, exp_r, p_win):
    b = exp_r
    q = 1 - p_win
    f = (p_win * (b + 1) - 1) / b
    f = f.clip(lower=0, upper=0.5) if hasattr(f, 'clip') else max(0, min(f, 0.5))
    alloc = f * nav
    return (alloc / prices).astype(int)

kelly_pos = kelly(signals, prices, nav, exp_r, p_win)
kelly_pos

,close
timestamp,
1514764800,0
1514764860,0
1514764920,0
1514764980,0
1514765040,0
...,...
1735775700,0
1735775760,0
1735775820,0


## 7. Maximum Drawdown Control

Scale back size when current drawdown grows.

- **Pros**: protects in downturns  
- **Cons**: may lock in losses
- If the portfolio has fallen, we reduce position sizes to avoid making losses worse during tough periods.

In [ ]:
# dummy drawdown series
drawdown = pd.Series([0.0,0.05,0.10,0.02,0.12,0.08], index=dates)

def dd_control(signals, prices, nav, drawdown, max_dd=0.2):
    scale = 1 - (drawdown / max_dd)
    alloc = signals.mul(nav * 0.1 * scale, axis=0)
    return (alloc / prices).fillna(0).astype(int)

dd_pos = dd_control(signals, prices, nav, drawdown)
dd_pos

# Summary

- **Equal**: Easiest, ignores risk.  
- **Vol-Scaled**: Balances volatility.  
- **Notional**: Caps $% per trade.  
- **Fixed-Frac**: Caps $‑at‑risk (needs stops).  
- **Risk-Parity**: Equalises risk contribution.  
- **Kelly**: Maximises growth.  
- **Drawdown**: Pulls back in bad times.